In [ ]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import os, json
import kfp

In [ ]:
components_url = "https://raw.githubusercontent.com/oneconvergence/dkube/master/components/"
dkube_training_op = kfp.components.load_component_from_url(components_url + "training/component.yaml")
dkube_storage_op  = kfp.components.load_component_from_url(components_url + "storage/component.yaml")
dkube_submit_op = kfp.components.load_component_from_url(components_url + "submit/component.yaml")
runid = 0

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='example titanic pipeline to submit to leaderboard'
)
def titanic_pipeline(token, project_id):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu-multiuser:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="titanic", run_script="python train.py",
                                    datasets='["titanic-train","titanic-test"]', outputs='["titanic"]',
                                    input_dataset_mounts='["/titanic-train","/titanic-test"]',
                                    output_mounts='["/model"]',envs='[{"EPOCHS": "1"}]')


    with kfp.dsl.ExitHandler(exit_op=dkube_storage_op("reclaim", token, namespace="kubeflow",uid="{{workflow.uid}}")):
        input_volumes = json.dumps(["{{workflow.uid}}-dataset@dataset://ocdkube:titanic-test",
                                    "{{workflow.uid}}-model@model://titanic",
                                    "{{workflow.uid}}-code@program://titanic"
                                    ])
        storage  = dkube_storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes).after(train)

    
        predict_op = kfp.dsl.ContainerOp(
            name="predict",
            image="ocdr/dkube-datascience-tf-cpu:v2.0.0",
            command=["python", "/code/predict.py"],
            pvolumes={"/titanic-test/": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-dataset"),
                     "/model/": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-model"),
                     "/code/": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-code")
                     },
            file_outputs={"output": "/tmp/prediction.csv"},
        ).after(storage)

        predictions = kfp.dsl.InputArgumentPath(predict_op.outputs["output"])

        submit = dkube_submit_op(token, project_id, predictions=predict_op.outputs["output"]).after(predict_op)

In [ ]:
token  = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
#Replace [titanic] & wprz8s with your project name and id respectively.
client.create_run_from_pipeline_func(titanic_pipeline, run_name="[titanic] Run" + str(runid), arguments={"token":token,"project_id":"wprz8s"})
runid += 1